# Data Madness Notebook - "Go where you are treated best !" 

**The following notebook presents the work done by Group 2 for the Data Madness project which is part of the examination for the third year course KEN 3450 Data Analysis.** 

In [47]:
# Comment Hendrik : Say something about the permitted terms of distribution for this work.

In [194]:
# IMPORTS GALORE
import pandas as pd
import numpy as np
import scipy as sp

import statsmodels.formula.api as sm
from sklearn import linear_model

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')   #seaborn-poster
plt.rcParams['figure.figsize'] = (11, 8)


import requests 

#activate if need arises:
import warnings
warnings.filterwarnings('ignore')

## Part 1: EDA
### Import the  data from the two main sources:
+ [2017](https://www.kaggle.com/kaggle/kaggle-survey-2017), [2018](https://www.kaggle.com/kaggle/kaggle-survey-2018) and [2019](https://www.kaggle.com/c/kaggle-survey-2019/data) Kaggle Machine Learning & Data Science Survey **(1)**
+ 2017, 2018 and 2019 [Data Professional Salary Survey Results](https://www.brentozar.com/archive/2020/01/the-2020-data-professional-salary-survey-results-are-in/). The survey is conducted by Brent Ozar, a private database consultant. **(2)**

### 1.1.1 Import the ML and Data Science Survey Data

In [165]:
#local filepath, do not re-run cell or specify given file path on local machine 
file = '/Users/HendrikS/Documents/GitHub/Data-Madness/2017multipleChoiceResponses.csv'
import chardet
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

+ The data is in ascii encoding, thus the encoding that we use with the pandas `pandas.read_csv` function should be [ISO-8859-1](https://en.wikipedia.org/wiki/ISO/IEC_8859-1):

In [166]:
df_KGLresponse17 = pd.read_csv('2017multipleChoiceResponses.csv',encoding = "ISO-8859-1", engine ='python')
df_KGLresponse18 = pd.read_csv('2018multipleChoiceResponses.csv',encoding = "ISO-8859-1", engine ='python')
df_KGLresponse19 = pd.read_csv('2019multipleChoiceResponses.csv',encoding = "ISO-8859-1", engine ='python')

+ For the exploration and in order to spot similar variables between the different data frames, we want to see all variable Names:

In [167]:
pd.set_option('display.max_columns', None)

**Get an idea of the data of 2017:**

In [168]:
df_KGLresponse17.head(5)

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,CurrentEmployerType,MLToolNextYearSelect,MLMethodNextYearSelect,LanguageRecommendationSelect,PublicDatasetsSelect,LearningPlatformSelect,LearningPlatformUsefulnessArxiv,LearningPlatformUsefulnessBlogs,LearningPlatformUsefulnessCollege,LearningPlatformUsefulnessCompany,LearningPlatformUsefulnessConferences,LearningPlatformUsefulnessFriends,LearningPlatformUsefulnessKaggle,LearningPlatformUsefulnessNewsletters,LearningPlatformUsefulnessCommunities,LearningPlatformUsefulnessDocumentation,LearningPlatformUsefulnessCourses,LearningPlatformUsefulnessProjects,LearningPlatformUsefulnessPodcasts,LearningPlatformUsefulnessSO,LearningPlatformUsefulnessTextbook,LearningPlatformUsefulnessTradeBook,LearningPlatformUsefulnessTutoring,LearningPlatformUsefulnessYouTube,BlogsPodcastsNewslettersSelect,LearningDataScienceTime,JobSkillImportanceBigData,JobSkillImportanceDegree,JobSkillImportanceStats,JobSkillImportanceEnterpriseTools,JobSkillImportancePython,JobSkillImportanceR,JobSkillImportanceSQL,JobSkillImportanceKaggleRanking,JobSkillImportanceMOOC,JobSkillImportanceVisualizations,JobSkillImportanceOtherSelect1,JobSkillImportanceOtherSelect2,JobSkillImportanceOtherSelect3,CoursePlatformSelect,HardwarePersonalProjectsSelect,TimeSpentStudying,ProveKnowledgeSelect,DataScienceIdentitySelect,FormalEducation,MajorSelect,Tenure,PastJobTitlesSelect,FirstTrainingSelect,LearningCategorySelftTaught,LearningCategoryOnlineCourses,LearningCategoryWork,LearningCategoryUniversity,LearningCategoryKaggle,LearningCategoryOther,MLSkillsSelect,MLTechniquesSelect,ParentsEducation,EmployerIndustry,EmployerSize,EmployerSizeChange,EmployerMLTime,EmployerSearchMethod,UniversityImportance,JobFunctionSelect,WorkHardwareSelect,WorkDataTypeSelect,WorkProductionFrequency,WorkDatasetSize,WorkAlgorithmsSelect,WorkToolsSelect,WorkToolsFrequencyAmazonML,WorkToolsFrequencyAWS,WorkToolsFrequencyAngoss,WorkToolsFrequencyC,WorkToolsFrequencyCloudera,WorkToolsFrequencyDataRobot,WorkToolsFrequencyFlume,WorkToolsFrequencyGCP,WorkToolsFrequencyHadoop,WorkToolsFrequencyIBMCognos,WorkToolsFrequencyIBMSPSSModeler,WorkToolsFrequencyIBMSPSSStatistics,WorkToolsFrequencyIBMWatson,WorkToolsFrequencyImpala,WorkToolsFrequencyJava,WorkToolsFrequencyJulia,WorkToolsFrequencyJupyter,WorkToolsFrequencyKNIMECommercial,WorkToolsFrequencyKNIMEFree,WorkToolsFrequencyMathematica,WorkToolsFrequencyMATLAB,WorkToolsFrequencyAzure,WorkToolsFrequencyExcel,WorkToolsFrequencyMicrosoftRServer,WorkToolsFrequencyMicrosoftSQL,WorkToolsFrequencyMinitab,WorkToolsFrequencyNoSQL,WorkToolsFrequencyOracle,WorkToolsFrequencyOrange,WorkToolsFrequencyPerl,WorkToolsFrequencyPython,WorkToolsFrequencyQlik,WorkToolsFrequencyR,WorkToolsFrequencyRapidMinerCommercial,WorkToolsFrequencyRapidMinerFree,WorkToolsFrequencySalfrod,WorkToolsFrequencySAPBusinessObjects,WorkToolsFrequencySASBase,WorkToolsFrequencySASEnterprise,WorkToolsFrequencySASJMP,WorkToolsFrequencySpark,WorkToolsFrequencySQL,WorkToolsFrequencyStan,WorkToolsFrequencyStatistica,WorkToolsFrequencyTableau,WorkToolsFrequencyTensorFlow,WorkToolsFrequencyTIBCO,WorkToolsFrequencyUnix,WorkToolsFrequencySelect1,WorkToolsFrequencySelect2,WorkFrequencySelect3,WorkMethodsSelect,WorkMethodsFrequencyA/B,WorkMethodsFrequencyAssociationRules,WorkMethodsFrequencyBayesian,WorkMethodsFrequencyCNNs,WorkMethodsFrequencyCollaborativeFiltering,WorkMethodsFrequencyCross-Validation,WorkMethodsFrequencyDataVisualization,WorkMethodsFrequencyDecisionTrees,WorkMethodsFrequencyEnsembleMethods,WorkMethodsFrequencyEvolutionaryApproaches,WorkMethodsFrequencyGANs,WorkMethodsFrequencyGBM,WorkMethodsFrequencyHMMs,WorkMethodsFrequencyKNN,WorkMethodsFrequencyLiftAnalysis,WorkMethodsFrequencyLogisticRegression,WorkMethodsFrequencyMLN,WorkMethodsFrequencyNaiveBayes,WorkMethodsFrequencyNLP,WorkMethodsFrequencyNeuralNetworks,WorkMethodsFrequencyPCA,WorkMethodsFrequencyPrescripti

In [169]:
df_KGLresponse17.shape

(16716, 228)

+ Adding a `SurveyYear` column as the first column to all KGL dataframes:

In [170]:
# run this only once!
#2017
df_KGLresponse17['Survey Year'] = 2017
cols = df_KGLresponse17.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_KGLresponse17 = df_KGLresponse17[cols]
#2018
df_KGLresponse18['Survey Year'] = 2018
cols = df_KGLresponse18.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_KGLresponse18 = df_KGLresponse18[cols]
#2019
df_KGLresponse19['Survey Year'] = 2019
cols = df_KGLresponse19.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_KGLresponse19 = df_KGLresponse19[cols]


**Columns from KGL17 which are interesting for this analysis:**<br>

Columns that are interesting, but are difficult to work with  especially when considering the other data source (because e.g. the question goes into depth about ML) are noted in brackets. We have to keep in mind that the survey by Brent Ozar is answered by mainly database professionals which do not use, for instance, ML techniques often.

+ Column names are mostly self-explanatory. In case they are not, a description is added

+ `Survey Year`
+ `GenderSelect`
+ `Country` - The country you currently live in. (Note: As opposed to working in.)
+ `Age`  (not asked in other data source)
+ `EmploymentStatus`
+ `CurrentJobTitleSelect` (For job title there is also a FreeForm, i.e. the user can specify an own JobTitle which is not considered for this analysis). BUT: Free form responses are randomized within a column, so that reading across a single row does not give a single user's answers. Therefore, we disregard Free Form responses in this analysis.
+ `FormalEducation`
+ `MajorSelect` - Which best describes your undergraduate major? 
+ `Tenure` - How long have you been writing code to analyze data?
+ `EmployerSize` - How many employees work at your current or most recent company?
+ `CompensationAmount` - What is your current total yearly compensation (salary + bonus)? - Total Amount (e.g. 75,000)
+ `CompensationCurrency` - What is your current total yearly compensation (salary + bonus)? - Choose your currency
+ `JobSatisfaction` - On a scale from 0 (Highly Dissatisfied) - 10 (Highly Satisfied), how satisfied are you with your current job?


Possible additional variables:
+ (`MLSikllsSelect`) - In which areas of machine learning do you consider yourself competent?
+ (`MLTechniquesSelect`) - In which machine learning techniques do you consider yourself competent?
+ `ParentsEducation` - What's the highest level of education completed by either of your parents?
+ `EmployerIndustry` (no comparable variable in other data source, only `EmploymentSector`-public or private)

+ `JobFactorSalary` - How are you assessing potential job opportunities? - The compensation and benefits offered   (only asked to Learners)
+ `JobFactor_X_` - JobFactor questions only asked to respondents of type 'Learners'



**Get an idea of the data of 2018:**

In [171]:
df_KGLresponse18.head()  #header should be =1

,Survey Year,Time from Start to Finish (seconds),Q1,Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,Q7_OTHER_TEXT,Q8,Q9,Q10,Q11_Part_1,Q11_Part_2,Q11_Part_3,Q11_Part_4,Q11_Part_5,Q11_Part_6,Q11_Part_7,Q11_OTHER_TEXT,Q12_MULTIPLE_CHOICE,Q12_Part_1_TEXT,Q12_Part_2_TEXT,Q12_Part_3_TEXT,Q12_Part_4_TEXT,Q12_Part_5_TEXT,Q12_OTHER_TEXT,Q13_Part_1,Q13_Part_2,Q13_Part_3,Q13_Part_4,Q13_Part_5,Q13_Part_6,Q13_Part_7,Q13_Part_8,Q13_Part_9,Q13_Part_10,Q13_Part_11,Q13_Part_12,Q13_Part_13,Q13_Part_14,Q13_Part_15,Q13_OTHER_TEXT,Q14_Part_1,Q14_Part_2,Q14_Part_3,Q14_Part_4,Q14_Part_5,Q14_Part_6,Q14_Part_7,Q14_Part_8,Q14_Part_9,Q14_Part_10,Q14_Part_11,Q14_OTHER_TEXT,Q15_Part_1,Q15_Part_2,Q15_Part_3,Q15_Part_4,Q15_Part_5,Q15_Part_6,Q15_Part_7,Q15_OTHER_TEXT,Q16_Part_1,Q16_Part_2,Q16_Part_3,Q16_Part_4,Q16_Part_5,Q16_Part_6,Q16_Part_7,Q16_Part_8,Q16_Part_9,Q16_Part_10,Q16_Part_11,Q16_Part_12,Q16_Part_13,Q16_Part_14,Q16_Part_15,Q16_Part_16,Q16_Part_17,Q16_Part_18,Q16_OTHER_TEXT,Q17,Q17_OTHER_TEXT,Q18,Q18_OTHER_TEXT,Q19_Part_1,Q19_Part_2,Q19_Part_3,Q19_Part_4,Q19_Part_5,Q19_Part_6,Q19_Part_7,Q19_Part_8,Q19_Part_9,Q19_Part_10,Q19_Part_11,Q19_Part_12,Q19_Part_13,Q19_Part_14,Q19_Part_15,Q19_Part_16,Q19_Part_17,Q19_Part_18,Q19_Part_19,Q19_OTHER_TEXT,Q20,Q20_OTHER_TEXT,Q21_Part_1,Q21_Part_2,Q21_Part_3,Q21_Part_4,Q21_Part_5,Q21_Part_6,Q21_Part_7,Q21_Part_8,Q21_Part_9,Q21_Part_10,Q21_Part_11,Q21_Part_12,Q21_Part_13,Q21_OTHER_TEXT,Q22,Q22_OTHER_TEXT,Q23,Q24,Q25,Q26,Q27_Part_1,Q27_Part_2,Q27_Part_3,Q27_Part_4,Q27_Part_5,Q27_Part_6,Q27_Part_7,Q27_Part_8,Q27_Part_9,Q27_Part_10,Q27_Part_11,Q27_Part_12,Q27_Part_13,Q27_Part_14,Q27_Part_15,Q27_Part_16,Q27_Part_17,Q27_Part_18,Q27_Part_19,Q27_Part_20,Q27_OTHER_TEXT,Q28_Part_1,Q28_Part_2,Q28_Part_3,Q28_Part_4,Q28_Part_5,Q28_Part_6,Q28_Part_7,Q28_Part_8,Q28_Part_9,Q28_Part_10,Q28_Part_11,Q28_Part_12,Q28_Part_13,Q28_Part_14,Q28_Part_15,Q28_Part_16,Q28_Part_17,Q28_Part_18,Q28_Part_19,Q28_Part_20,Q28_Part_21,Q28_Part_22,Q28_Part_23,Q28_Part_24,Q28_Part_25,Q28_Part_26,Q28_Part_27,Q28_Part_28,Q28_Part_29,Q28_Part_30,Q28_Part_31,Q28_Part_32,Q28_Part_33,Q28_Part_34,Q28_Part_35,Q28_Part_36,Q28_Part_37,Q28_Part_38,Q28_Part_39,Q28_Part_40,Q28_Part_41,Q28_Part_42,Q28_Part_43,Q28_OTHER_TEXT,Q29_Part_1,Q29_Part_2,Q29_Part_3,Q29_Part_4,Q29_Part_5,Q29_Part_6,Q29_Part_7,Q29_Part_8,Q29_Part_9,Q29_Part_10,Q29_Part_11,Q29_Part_12,Q29_Part_13,Q29_Part_14,Q29_Part_15,Q29_Part_16,Q29_Part_17,Q29_Part_18,Q29_Part_19,Q29_Part_20,Q29_Part_21,Q29_Part_22,Q29_Part_23,Q29_Part_24,Q29_Part_25,Q29_Part_26,Q29_Part_27,Q29_Part_28,Q29_OTHER_TEXT,Q30_Part_1,Q30_Part_2,Q30_Part_3,Q30_Part_4,Q30_Part_5,Q30_Part_6,Q30_Part_7,Q30_Part_8,Q30_Part_9,Q30_Part_10,Q30_Part_11,Q30_Part_12,Q30_Part_13,Q30_Part_14,Q30_Part_15,Q30_Part_16,Q30_Part_17,Q30_Part_18,Q30_Part_19,Q30_Part_20,Q30_Part_21,Q30_Part_22,Q30_Part_23,Q30_Part_24,Q30_Part_25,Q30_OTHER_TEXT,Q31_Part_1,Q31_Part_2,Q31_Part_3,Q31_Part_4,Q31_Part_5,Q31_Part_6,Q31_Part_7,Q31_Part_8,Q31_Part_9,Q31_Part_10,Q31_Part_11,Q31_Part_12,Q31_OTHER_TEXT,Q32,Q32_OTHER,Q33_Part_1,Q33_Part_2,Q33_Part_3,Q33_Part_4,Q33_Part_5,Q33_Part_6,Q33_Part_7,Q33_Part_8,Q33_Part_9,Q33_Part_10,Q33_Part_11,Q33_OTHER_TEXT,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,Q34_OTHER_TEXT,Q35_Part_1,Q35_Part_2,Q35_Part_3,Q35_Part_4,Q35_Part_5,Q35_Part_6,Q35_OTHER_TEXT,Q36_Part_1,Q36_Part_2,Q36_Part_3,Q36_Part_4,Q36_Part_5,Q36_Part_6,Q36_Part_7,Q36_Part_8,Q36_Part_9,Q36_Part_10,Q36_Part_11,Q36_Part_12,Q36_Part_13,Q36_OTHER_TEXT,Q37,Q37_OTHER_TEXT,Q38_Part_1,Q38_Part_2,Q38_Part_3,Q38_Part_4,Q38_Part_5,Q38_Part_6,Q38_Part_7,Q38_Part_8,Q38_Part_9,Q38_Part_10,Q38_Part_11,Q38_Part_12,Q38_Part_13,Q38_Part_14,Q38_Part_15,Q38_Part_16,Q38_Part_17,Q38_Part_18,Q38_Part_19,Q38_Part_20,Q38_Part_21,Q38_Part_22,Q38_OTHER_TEXT,Q39_Part_1,Q39_Part_2,Q40,Q41_Part_1,Q41_Part_2,Q41_Part_3,Q42_Part_1,Q42_Part_2,Q42_Part_3,Q42_Part_4,Q42_Part_5,Q42_OTHER_TEXT,Q43,Q44_Part_1,Q44_Part_2,Q44_Part_3,Q44_Part_4,Q44_Part_5,Q44_Part_6,Q45_Part_1,Q45_

In [172]:
df_KGLresponse18.shape

(23860, 396)

+ For the 2018 and 2019 survey the `CompensationAmount` is asked for as follows: What is your current yearly compensation (approximate $USD)? So we do not have an additional currency column.

+ From 2017 to 2018 we are loosing following variables: `EmploymentStatus`,`EmployerSize`,`JobSatisfaction`

+ Renaming the relevant columns in the 2018 data:

In [173]:
df_KGLresponse18 = df_KGLresponse18.rename(columns={'Q1': 'GenderSelect', 'Q2': 'Age', 'Q3': 'Country', 'Q4': 'FormalEducation', 'Q6': 'CurrentJobTitleSelect', 'Q7' : 'EmployerIndustry', 'Q8' : 'Tenure', 'Q9' : 'SalaryUSD'})                              


**Get an idea of the data of 2019:**

In [119]:
df_KGLresponse19.head(2)  #header should be =1

,Survey Year,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,Q8,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,Q9_Part_6,Q9_Part_7,Q9_Part_8,Q9_OTHER_TEXT,Q10,Q11,Q12_Part_1,Q12_Part_2,Q12_Part_3,Q12_Part_4,Q12_Part_5,Q12_Part_6,Q12_Part_7,Q12_Part_8,Q12_Part_9,Q12_Part_10,Q12_Part_11,Q12_Part_12,Q12_OTHER_TEXT,Q13_Part_1,Q13_Part_2,Q13_Part_3,Q13_Part_4,Q13_Part_5,Q13_Part_6,Q13_Part_7,Q13_Part_8,Q13_Part_9,Q13_Part_10,Q13_Part_11,Q13_Part_12,Q13_OTHER_TEXT,Q14,Q14_Part_1_TEXT,Q14_Part_2_TEXT,Q14_Part_3_TEXT,Q14_Part_4_TEXT,Q14_Part_5_TEXT,Q14_OTHER_TEXT,Q15,Q16_Part_1,Q16_Part_2,Q16_Part_3,Q16_Part_4,Q16_Part_5,Q16_Part_6,Q16_Part_7,Q16_Part_8,Q16_Part_9,Q16_Part_10,Q16_Part_11,Q16_Part_12,Q16_OTHER_TEXT,Q17_Part_1,Q17_Part_2,Q17_Part_3,Q17_Part_4,Q17_Part_5,Q17_Part_6,Q17_Part_7,Q17_Part_8,Q17_Part_9,Q17_Part_10,Q17_Part_11,Q17_Part_12,Q17_OTHER_TEXT,Q18_Part_1,Q18_Part_2,Q18_Part_3,Q18_Part_4,Q18_Part_5,Q18_Part_6,Q18_Part_7,Q18_Part_8,Q18_Part_9,Q18_Part_10,Q18_Part_11,Q18_Part_12,Q18_OTHER_TEXT,Q19,Q19_OTHER_TEXT,Q20_Part_1,Q20_Part_2,Q20_Part_3,Q20_Part_4,Q20_Part_5,Q20_Part_6,Q20_Part_7,Q20_Part_8,Q20_Part_9,Q20_Part_10,Q20_Part_11,Q20_Part_12,Q20_OTHER_TEXT,Q21_Part_1,Q21_Part_2,Q21_Part_3,Q21_Part_4,Q21_Part_5,Q21_OTHER_TEXT,Q22,Q23,Q24_Part_1,Q24_Part_2,Q24_Part_3,Q24_Part_4,Q24_Part_5,Q24_Part_6,Q24_Part_7,Q24_Part_8,Q24_Part_9,Q24_Part_10,Q24_Part_11,Q24_Part_12,Q24_OTHER_TEXT,Q25_Part_1,Q25_Part_2,Q25_Part_3,Q25_Part_4,Q25_Part_5,Q25_Part_6,Q25_Part_7,Q25_Part_8,Q25_OTHER_TEXT,Q26_Part_1,Q26_Part_2,Q26_Part_3,Q26_Part_4,Q26_Part_5,Q26_Part_6,Q26_Part_7,Q26_OTHER_TEXT,Q27_Part_1,Q27_Part_2,Q27_Part_3,Q27_Part_4,Q27_Part_5,Q27_Part_6,Q27_OTHER_TEXT,Q28_Part_1,Q28_Part_2,Q28_Part_3,Q28_Part_4,Q28_Part_5,Q28_Part_6,Q28_Part_7,Q28_Part_8,Q28_Part_9,Q28_Part_10,Q28_Part_11,Q28_Part_12,Q28_OTHER_TEXT,Q29_Part_1,Q29_Part_2,Q29_Part_3,Q29_Part_4,Q29_Part_5,Q29_Part_6,Q29_Part_7,Q29_Part_8,Q29_Part_9,Q29_Part_10,Q29_Part_11,Q29_Part_12,Q29_OTHER_TEXT,Q30_Part_1,Q30_Part_2,Q30_Part_3,Q30_Part_4,Q30_Part_5,Q30_Part_6,Q30_Part_7,Q30_Part_8,Q30_Part_9,Q30_Part_10,Q30_Part_11,Q30_Part_12,Q30_OTHER_TEXT,Q31_Part_1,Q31_Part_2,Q31_Part_3,Q31_Part_4,Q31_Part_5,Q31_Part_6,Q31_Part_7,Q31_Part_8,Q31_Part_9,Q31_Part_10,Q31_Part_11,Q31_Part_12,Q31_OTHER_TEXT,Q32_Part_1,Q32_Part_2,Q32_Part_3,Q32_Part_4,Q32_Part_5,Q32_Part_6,Q32_Part_7,Q32_Part_8,Q32_Part_9,Q32_Part_10,Q32_Part_11,Q32_Part_12,Q32_OTHER_TEXT,Q33_Part_1,Q33_Part_2,Q33_Part_3,Q33_Part_4,Q33_Part_5,Q33_Part_6,Q33_Part_7,Q33_Part_8,Q33_Part_9,Q33_Part_10,Q33_Part_11,Q33_Part_12,Q33_OTHER_TEXT,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT
0,2019,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,Select the title most similar to your current ...,What is the size of the company where you are ...,Approximately how many individuals are respons...,Does your current employer incorporate machine...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,What is your current yearly compensation (appr...,Approximately how much money have you spent on...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media 

In [175]:
df_KGLresponse19.shape

(19718, 247)

+ `EmployerSize` is again a variable in the 2019 data, instead `EmployerIndustry` is missing.

Renaming the relevant columns in the 2019 data:

In [176]:
df_KGLresponse19 = df_KGLresponse19.rename(columns={'Q1': 'Age', 'Q2': 'GenderSelect', 'Q3': 'Country', 'Q4': 'FormalEducation', 'Q5': 'CurrentJobTitleSelect', 'Q6' : 'EmployerSize', 'Q10' : 'SalaryUSD', 'Q15' : 'Tenure'}) 


For 2018 and 2019 the order of questions has changed quite a bit. Even from 2018 to 2019 there are quite some changes. in Section 1.1.3 we extract the relevant columns from all three data frames.

### 1.1.2 Import the Professional Salary Survey Data

In [177]:
df_OZRresponse = pd.read_excel('Data_Professional_Salary_Survey_Responses.xlsx', header = 3)

In [178]:
df_OZRresponse.head()

,Survey Year,Timestamp,SalaryUSD,Country,PostalCode,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,ManageStaff,YearsWithThisTypeOfJob,HowManyCompanies,OtherPeopleOnYourTeam,CompanyEmployeesOverall,DatabaseServers,Education,EducationIsComputerRelated,Certifications,HoursWorkedPerWeek,TelecommuteDaysPerWeek,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed,Counter
0,2017,2017-01-05 05:10:20.451,200000,United States,Not Asked,Microsoft SQL Server,10,MySQL/MariaDB,Full time employee,DBA,No,5,Not Asked,2,Not Asked,350,Masters,No,"Yes, and they're currently valid",45,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
1,2017,2017-01-05 05:26:23.388,61515,United Kingdom,Not Asked,Microsoft SQL Server,15,"Oracle, PostgreSQL",Full time employee,DBA,No,3,Not Asked,1,Not Asked,40,None (no degree completed),NaN,"No, I never have",35,2,Not Asked,Private business,No,Not Asked,Not Asked,Not Asked,Not Asked,1
2,2017,2017-01-05 05:32:57.367,95000,Germany,Not Asked,Microsoft SQL Server,5,"Oracle, MySQL/MariaDB, Informix",Full time employee,Other,Yes,25,Not Asked,2,Not Asked,100,Masters,Yes,"Yes, but they expired",45,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
3,2017,2017-01-05 05:33:03.316,56000,United Kingdom,Not Asked,Microsoft SQL Server,6,NaN,Full time employee,DBA,No,2,Not Asked,None,Not Asked,500,Associates (2 years),No,"No, I never have",40,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
4,2017,2017-01-05 05:34:33.866,35000,France,Not Asked,Microsoft SQL Server,10,Oracle,Full time employee of a consulting/contracting...,DBA,No,10,Not Asked,None,Not Asked,30,Bachelors (4 years),Yes,"Yes, but they expired",40,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1


Observation: Some questions are only asked up from more recent years. Most importantly, Gender was only asked up from 2018. 

**Columns which interesting for this analysis:**
+ `Survey Year`
+ `SalaryUSD`
+ `Country`  (We assume here (and it also follows from the [website](https://www.brentozar.com/archive/2020/01/the-2020-data-professional-salary-survey-results-are-in/) of the survey "[...] of you in 63 countries answered"), that this is the country of residence of the respondent.  Since this is the case for both data sources, we assume from now on that the country variable encodes the country of residence and that it is the same as the country where one is employed/works (which might not be true in all cases).
+ `EmploymentStatus`
+ `JobTitle`
+ `YearsWithThisTypeOfJob`
+ `CompanyEmployeesOverall` (was only asked up from 2019)
+ `Education`
+ `HoursWorkedPerWeek` (not a variable in other data source)
+ `PopulationOfLargestCityWithin20Miles` (was only asked up from 2019 and no comparable variable in other data source)
+ `LookingForAnotherJob` (somewhat comparable with `JobSatisfaction` from KGLE data)
+ `Gender` (was only asked up from 2018)



### 1.1.3 Extracting the relevant data

Let's extract the variables that are relevant for the data analysis from the first source, the Kaggle ML and Data Science Surveys from the years 2017, 2018 and 2019 first.The questions/variables have changed in content as well as location from year to year. We thus have to manually gather the variables from each dataframe that we are interested in.

In [179]:
# conflicts: age not in OZR Data, still need to deal with different currencies, deal with EmployerIndustry

relevantVars17 = ['Survey Year',
                  'CompensationAmount',
                  'CompensationCurrency',
                  'Country',
                  'EmploymentStatus',
                  'CurrentJobTitleSelect',
                  'Tenure',
                  'EmployerSize',
                  'FormalEducation',
                  'JobSatisfaction',
                  'EmployerIndustry',
                  'GenderSelect',
                  'Age']    #leave MajorSelect out for now (conflict with other dataset)

df_KGL17_da = df_KGLresponse17[relevantVars17]
df_KGL17_da

,Survey Year,CompensationAmount,CompensationCurrency,Country,EmploymentStatus,CurrentJobTitleSelect,Tenure,EmployerSize,FormalEducation,JobSatisfaction,EmployerIndustry,GenderSelect,Age
0,2017,NaN,NaN,NaN,Employed full-time,DBA/Database Engineer,More than 10 years,100 to 499 employees,Bachelor's degree,5,Internet-based,"Non-binary, genderqueer, or gender non-conforming",NaN
1,2017,NaN,NaN,United States,"Not employed, but looking for work",NaN,Less than a year,NaN,Master's degree,NaN,NaN,Female,30.0
2,2017,NaN,NaN,Canada,"Not employed, but looking for work",NaN,3 to 5 years,NaN,Master's degree,NaN,NaN,Male,28.0
3,2017,"250,000",USD,United States,"Independent contractor, freelancer, or self-em...",Operations Research Practitioner,More than 10 years,NaN,Master's degree,10 - Highly Satisfied,Mix of fields,Male,56.0
4,2017,NaN,NaN,Taiwan,Employed full-time,Computer Scientist,More than 10 years,"5,000 to 9,999 employees",Doctoral degree,2,Technology,Male,38.0
5,2017,NaN,NaN,Brazil,Employed full-time,Data Scientist,6 to 10 years,500 to 999 employees,Doctoral degree,8,Academic,Male,46.0
6,2017,NaN,NaN,United States,Employed full-time,Computer Scientist,More than 10 years,"10,000 or more employees",Master's degree,8,Government,Male,35.0
7,2017,NaN,NaN,India,Employed full-time,Software Developer/Software Engineer,1 to 2 years,NaN,Bachelor's degree,NaN,Technology,Female,22.0
8,2017,80000,AUD,Australia,Employed full-time,Business Analyst,3 to 5 years,20 to 99 employees,Bachelor's degree,7,Non-profit,Female,43.0
9,2017,1200000,RUB,Russia,Employed full-time,Software Developer/Software Engineer,1 to 2 years,20 to 99 employees,Bachelor's degree,7,Internet-based,Male,33.0


In [186]:
relevantVars18 = ['Survey Year',
                  'SalaryUSD',
                  'Country',
                  'CurrentJobTitleSelect',
                  'Tenure',
                  'FormalEducation',
                  'EmployerIndustry',
                  'GenderSelect',
                  'Age'] 

df_KGL18_da = df_KGLresponse18[relevantVars18]
df_KGL18_da

,Survey Year,SalaryUSD,Country,CurrentJobTitleSelect,Tenure,FormalEducation,EmployerIndustry,GenderSelect,Age
0,2018,What is your current yearly compensation (appr...,In which country do you currently reside?,Select the title most similar to your current ...,How many years of experience do you have in yo...,What is the highest level of formal education ...,In what industry is your current employer/cont...,What is your gender? - Selected Choice,What is your age (# years)?
1,2018,NaN,United States of America,Consultant,NaN,Doctoral degree,Other,Female,45-49
2,2018,"10-20,000",Indonesia,Other,5-10,Bachelorâs degree,Manufacturing/Fabrication,Male,30-34
3,2018,"0-10,000",United States of America,Data Scientist,0-1,Masterâs degree,I am a student,Female,30-34
4,2018,NaN,United States of America,Not employed,NaN,Masterâs degree,NaN,Male,35-39
5,2018,"0-10,000",India,Data Analyst,0-1,Masterâs degree,I am a student,Male,22-24
6,2018,I do not wish to disclose my approximate yearl...,Colombia,Data Scientist,0-1,Bachelorâs degree,Computers/Technology,Male,25-29
7,2018,"10-20,000",Chile,Other,10-15,Doctoral degree,Academics/Education,Male,35-39
8,2018,"0-10,000",India,Other,0-1,Masterâs degree,Other,Male,18-21
9,2018,NaN,Turkey,Not employed,NaN,Masterâs degree,NaN,Male,25-29


In [185]:
df_KGLresponse19 = df_KGLresponse19.rename(columns={'Q1': 'Age', 'Q2': 'GenderSelect', 'Q3': 'Country', 'Q4': 'FormalEducation', 'Q5': 'CurrentJobTitleSelect', 'Q6' : 'EmployerSize', 'Q10' : 'SalaryUSD', 'Q15' : 'Tenure'})
relevantVars19 = ['Survey Year',
                  'SalaryUSD',
                  'Country',
                  'CurrentJobTitleSelect',
                  'Tenure',
                  'FormalEducation',
                  'EmployerSize',
                  'GenderSelect',
                  'Age'] 

df_KGL19_da = df_KGLresponse19[relevantVars19]
df_KGL19_da

,Survey Year,SalaryUSD,Country,CurrentJobTitleSelect,Tenure,FormalEducation,EmployerSize,GenderSelect,Age
0,2019,What is your current yearly compensation (appr...,In which country do you currently reside?,Select the title most similar to your current ...,How long have you been writing code to analyze...,What is the highest level of formal education ...,What is the size of the company where you are ...,What is your gender? - Selected Choice,What is your age (# years)?
1,2019,"30,000-39,999",France,Software Engineer,1-2 years,Masterâs degree,"1000-9,999 employees",Male,22-24
2,2019,"5,000-7,499",India,Software Engineer,I have never written code,Professional degree,"> 10,000 employees",Male,40-44
3,2019,NaN,Germany,NaN,NaN,Professional degree,NaN,Female,55-59
4,2019,"250,000-299,999",Australia,Other,1-2 years,Masterâs degree,"> 10,000 employees",Male,40-44
5,2019,"4,000-4,999",India,Other,< 1 years,Bachelorâs degree,0-49 employees,Male,22-24
6,2019,"60,000-69,999",France,Data Scientist,20+ years,Masterâs degree,0-49 employees,Male,50-54
7,2019,"10,000-14,999",India,Data Scientist,3-5 years,Masterâs degree,50-249 employees,Male,22-24
8,2019,"80,000-89,999",United States of America,Data Scientist,3-5 years,Bachelorâs degree,"> 10,000 employees",Female,22-24
9,2019,NaN,United States of America,Student,3-5 years,Bachelorâs degree,NaN,Male,22-24


**Assumptions:**
+ `Tenure`(How long have you been writing code to analyze data?) and `YearsWithinThisTypeOfJob` are comparable variables. Although they might not capture the exact same meaning, they still have large contextual intersection. In reality, we would perhaps expect `Tenure` to be slightly longer than `YearsWithinThisTypeOfJob` for a given person. However, in most cases `YearsWithinThisTypeOfJob` cannot be shorter than Tenure, i.e. one needs to be able to write code to work as a (DB) engineer in the Information Sector.

In [183]:
relevantVarsOZR = ['Survey Year',
                  'SalaryUSD',
                  'Country',
                  'EmploymentStatus',
                  'JobTitle',
                  'YearsWithThisTypeOfJob',
                  'CompanyEmployeesOverall',
                  'Education',
                  'LookingForAnotherJob',
                  'Gender']  # leave HoursWorkedPerWeek,PopulationOfLargestCityWithin20Miles out for now (conflict with other dataset)
df_OZRresponse_da = df_OZRresponse[relevantVarsOZR]

In [135]:
df_OZRresponse_filter.shape

(6893, 10)

In [207]:
df_KGL17_da

,Survey Year,CompensationAmount,CompensationCurrency,Country,EmploymentStatus,CurrentJobTitleSelect,Tenure,EmployerSize,FormalEducation,JobSatisfaction,EmployerIndustry,GenderSelect,Age
0,2017,NaN,NaN,NaN,Employed full-time,DBA/Database Engineer,More than 10 years,100 to 499 employees,Bachelor's degree,5,Internet-based,"Non-binary, genderqueer, or gender non-conforming",NaN
1,2017,NaN,NaN,United States,"Not employed, but looking for work",NaN,Less than a year,NaN,Master's degree,NaN,NaN,Female,30.0
2,2017,NaN,NaN,Canada,"Not employed, but looking for work",NaN,3 to 5 years,NaN,Master's degree,NaN,NaN,Male,28.0
3,2017,250000,USD,United States,"Independent contractor, freelancer, or self-em...",Operations Research Practitioner,More than 10 years,NaN,Master's degree,10 - Highly Satisfied,Mix of fields,Male,56.0
4,2017,NaN,NaN,Taiwan,Employed full-time,Computer Scientist,More than 10 years,"5,000 to 9,999 employees",Doctoral degree,2,Technology,Male,38.0
5,2017,NaN,NaN,Brazil,Employed full-time,Data Scientist,6 to 10 years,500 to 999 employees,Doctoral degree,8,Academic,Male,46.0
6,2017,NaN,NaN,United States,Employed full-time,Computer Scientist,More than 10 years,"10,000 or more employees",Master's degree,8,Government,Male,35.0
7,2017,NaN,NaN,India,Employed full-time,Software Developer/Software Engineer,1 to 2 years,NaN,Bachelor's degree,NaN,Technology,Female,22.0
8,2017,80000,AUD,Australia,Employed full-time,Business Analyst,3 to 5 years,20 to 99 employees,Bachelor's degree,7,Non-profit,Female,43.0
9,2017,1200000,RUB,Russia,Employed full-time,Software Developer/Software Engineer,1 to 2 years,20 to 99 employees,Bachelor's degree,7,Internet-based,Male,33.0


Compute the salary in USD for the 2017 respondents. In order to do so, we must draw from the conversion rates for some point in time in 2017. According to the website, the survey was live from August 7th to August 25th. Therefore, lets take the conversion rates from end of August 2017.

In [203]:
df_KGL17_da.loc[:,'CompensationAmount']=df_KGL17_da.loc[:,'CompensationAmount'].str.replace('-','')
df_KGL17_da.loc[:,'CompensationAmount']=df_KGL17_da.loc[:,'CompensationAmount'].str.replace(',','')
df_KGL17_da.loc[:,'CompensationAmount']=df_KGL17_da.loc[:,'CompensationAmount'].str.replace("'",'')

Conv_rates = pd.read_csv('conversionRates17.csv')
Conv_rates.drop('Unnamed: 0',axis=1,inplace=True)

salary = df_KGL17_da[['CompensationAmount','CompensationCurrency','GenderSelect','Country','CurrentJobTitleSelect']]

salary = salary.merge(Conv_rates,left_on='CompensationCurrency',right_on='originCountry',how='left')
salary['Salary']=pd.to_numeric(salary['CompensationAmount'])*salary['exchangeRate']



In [213]:
df_KGL17_da = df_KGL17_da.merge(Conv_rates,left_on='CompensationCurrency',right_on='originCountry',how='left')
df_KGL17_da['SalaryUSD'] = pd.to_numeric(salary['CompensationAmount'])*salary['exchangeRate']

,Survey Year,CompensationAmount,CompensationCurrency,Country,EmploymentStatus,CurrentJobTitleSelect,Tenure,EmployerSize,FormalEducation,JobSatisfaction,EmployerIndustry,GenderSelect,Age,originCountry_x,exchangeRate_x,originCountry_y,exchangeRate_y,originCountry_x,exchangeRate_x,originCountry_y,exchangeRate_y,SalaryUSD,originCountry,exchangeRate
0,2017,NaN,NaN,NaN,Employed full-time,DBA/Database Engineer,More than 10 years,100 to 499 employees,Bachelor's degree,5,Internet-based,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250000.000,NaN,NaN
1,2017,NaN,NaN,United States,"Not employed, but looking for work",NaN,Less than a year,NaN,Master's degree,NaN,NaN,Female,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64184.800,NaN,NaN
2,2017,NaN,NaN,Canada,"Not employed, but looking for work",NaN,3 to 5 years,NaN,Master's degree,NaN,NaN,Male,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20882.400,NaN,NaN
3,2017,250000,USD,United States,"Independent contractor, freelancer, or self-em...",Operations Research Practitioner,More than 10 years,NaN,Master's degree,10 - Highly Satisfied,Mix of fields,Male,56.0,USD,1.000000,USD,1.000000,USD,1.000000,USD,1.000000,1483.900,USD,1.000000
4,2017,NaN,NaN,Taiwan,Employed full-time,Computer Scientist,More than 10 years,"5,000 to 9,999 employees",Doctoral degree,2,Technology,Male,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36634.400,NaN,NaN
5,2017,NaN,NaN,Brazil,Employed full-time,Data Scientist,6 to 10 years,500 to 999 employees,Doctoral degree,8,Academic,Male,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20000.000,NaN,NaN
6,2017,NaN,NaN,United States,Employed full-time,Computer Scientist,More than 10 years,"10,000 or more employees",Master's degree,8,Government,Male,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100000.000,NaN,NaN
7,2017,NaN,NaN,India,Employed full-time,Software Developer/Software Engineer,1 to 2 years,NaN,Bachelor's degree,NaN,Technology,Female,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,916.400,NaN,NaN
8,2017,80000,AUD,Australia,Employed full-time,Business Analyst,3 to 5 years,20 to 99 employees,Bachelor's degree,7,Non-profit,Female,43.0,AUD,0.802310,AUD,0.802310,AUD,0.802310,AUD,0.802310,10858.848,AUD,0.802310
9,2017,1200000,RUB,Russia,Employed full-time,Software Developer/Software Engineer,1 to 2 years,20 to 99 employees,Bachelor's degree,7,Internet-based,Male,33.0,RUB,0.017402,RUB,0.017402,RUB,0.017402,RUB,0.017402,53352.000,RUB,0.017402


In [206]:
salary

,CompensationAmount,CompensationCurrency,GenderSelect,Country,CurrentJobTitleSelect,originCountry,exchangeRate,Salary
0,250000,USD,Male,United States,Operations Research Practitioner,USD,1.000000,250000.0000
1,80000,AUD,Female,Australia,Business Analyst,AUD,0.802310,64184.8000
2,1200000,RUB,Male,Russia,Software Developer/Software Engineer,RUB,0.017402,20882.4000
3,95000,INR,Male,India,Data Scientist,INR,0.015620,1483.9000
4,1100000,TWD,Male,Taiwan,Software Developer/Software Engineer,TWD,0.033304,36634.4000
5,20000,USD,Male,United States,Researcher,USD,1.000000,20000.0000
6,100000,USD,Male,United States,Scientist/Researcher,USD,1.000000,100000.0000
7,20000,CZK,Male,Czech Republic,Other,CZK,0.045820,916.4000
8,624000,RUB,Male,Russia,Data Analyst,RUB,0.017402,10858.8480
9,156000000,COP,Male,Colombia,Data Scientist,COP,0.000342,53352.0000
